## 1. Import Libraries

In [101]:
import os
from tensorflow.keras.datasets import mnist
import numpy as np

## 2. Set Global Parameters

In [111]:
HIDDEN_LAYER_NEURONS = 10
OUTPUT_LAYER_NEURONS = 10

input_file_path = "../Training/model_weights.txt"  # Update this path to your weights file
output_directory = "./Hidden_Neuron_Weights/"  # Update this path to your desired output directory

dataWidth = 16
weightIntWidth = 3

outPATH = "Binary_Rep/"
inputFile = "Hidden_Neuron_Weights/"

mnist_input_file_path = 'Input_Bin/mnist_normalized_line_by_line.txt'
mnist_output_txt_path = 'Input_Bin/bin_mnist_normalized_line_by_line.txt'
mnist_output_bat_path = 'Input_Bin/bin_mnist_normalized_line_by_line.bat'

weightFracWidth = dataWidth - weightIntWidth
print(weightFracWidth)

13


## 3. Utils Function for Separate All of Weights

In [112]:
def parse_weight_line(line):
    """
    Parse a line from the weight file and extract the layer, source neuron,
    destination neuron, and weight value. Adjusted to handle commas.
    """
    parts = line.replace(',', '').split()  # Remove commas before splitting
    layer = int(parts[2])
    source_neuron = int(parts[4])
    dest_neuron = int(parts[9])
    weight = float(parts[-1])
    return layer, source_neuron, dest_neuron, weight

In [113]:
def distribute_weights(input_file, output_dir):
    # Ensure the output directory exists
    if(not os.path.exists(output_dir)):
        os.makedirs(output_dir)
    
    # Adjust initialization to include neuron '0'
    neuron_weights = {'hidden': {i: [] for i in range(0, HIDDEN_LAYER_NEURONS + 1)},
                      'output': {i: [] for i in range(0, OUTPUT_LAYER_NEURONS + 1)}}
    
    # Read and parse the input file
    with open(input_file, 'r') as f:
        for line in f:
            layer, source_neuron, dest_neuron, weight = parse_weight_line(line)       
            neuron_type = 'hidden' if layer == 1 else 'output'
            neuron_weights[neuron_type][dest_neuron].append((source_neuron, weight))
    
    # Write weights to separate files
    for neuron_type, neurons in neuron_weights.items():
        for neuron_id, weights in neurons.items():
            file_name = f"{neuron_type}_neuron_{neuron_id}_weights.txt"
            file_path = os.path.join(output_dir, file_name)
            with open(file_path, 'w') as f:
                for source_neuron, weight in weights:
                    f.write(f"{weight}\n")

## 4. Create Spare Weights File

In [114]:
distribute_weights(input_file_path, output_directory)

## 5. Convert Weights to the Binary Representation

### 5.1. To Binary Function

In [115]:
def DtoB(num, dataWidth, fracBits):
    if(num >= 0):
        num = num * (2 ** fracBits)
    else:
        num = (num + 2**(dataWidth + fracBits)) * (2 ** fracBits)
    num = int(num) & (2 ** dataWidth - 1)
    return format(num, '0{}b'.format(dataWidth))

In [116]:
files = [f for f in os.listdir(inputFile) if f.endswith('.txt')]
files

['hidden_neuron_0_weights.txt',
 'hidden_neuron_10_weights.txt',
 'hidden_neuron_1_weights.txt',
 'hidden_neuron_2_weights.txt',
 'hidden_neuron_3_weights.txt',
 'hidden_neuron_4_weights.txt',
 'hidden_neuron_5_weights.txt',
 'hidden_neuron_6_weights.txt',
 'hidden_neuron_7_weights.txt',
 'hidden_neuron_8_weights.txt',
 'hidden_neuron_9_weights.txt',
 'output_neuron_0_weights.txt',
 'output_neuron_10_weights.txt',
 'output_neuron_1_weights.txt',
 'output_neuron_2_weights.txt',
 'output_neuron_3_weights.txt',
 'output_neuron_4_weights.txt',
 'output_neuron_5_weights.txt',
 'output_neuron_6_weights.txt',
 'output_neuron_7_weights.txt',
 'output_neuron_8_weights.txt',
 'output_neuron_9_weights.txt']

### 5.2. Convert All Weights to Binary

In [117]:
for file in files:
    input_file_path = os.path.join(inputFile, file)
    output_file_path = os.path.join(outPATH + "txt/", f"bin_{file}")
    
    with open(input_file_path, 'r') as infile, open(output_file_path, 'w') as outfile:
        for line in infile:
            num = float(line.strip())  # Read and convert each line to a float
            binary_rep = DtoB(num, dataWidth, weightFracWidth)  # Convert to binary
            outfile.write(binary_rep + '\n')  # Write the binary representation to the output file

print("All files have been processed and saved in 'Binary_Rep/txt/' directory.")

All files have been processed and saved in 'Binary_Rep/txt/' directory.


### 5.3. Save Weights With `.bat` Format 

In [118]:
for file in files:
    input_file_path = os.path.join(inputFile, file)
    output_file_path = os.path.join(outPATH + "bat/", f"bin_{os.path.splitext(file)[0]}.bat")  # Save as .bat
    
    with open(input_file_path, 'r') as infile, open(output_file_path, 'w') as outfile:
        for line in infile:
            num = float(line.strip())  # Read and convert each line to a float
            binary_rep = DtoB(num, dataWidth, weightFracWidth)  # Convert to binary
            outfile.write(binary_rep + '\n')  # Write the binary representation to the output file

print("All files have been processed and saved in 'Binary_Rep/bat/' directory with .bat format.")

All files have been processed and saved in 'Binary_Rep/bat/' directory with .bat format.


## 6. Read 20 Test Input and Convert to The Binary

### 6.1. Read and Normalized

In [119]:
(x_train, y_train), (_, _) = mnist.load_data()

# Select the first 20 instances and normalize them
x_train_selected = x_train[:20] / 255.0

# Prepare the content to be saved in a text file
content = ""
for i in range(20):
    content += f"Image {i+1}: True class = {y_train[i]}\n"
    flattened = x_train_selected[i].flatten()  # Flatten the 28x28 image to a 1D array of 784 elements
    for pixel in flattened:
        content += f"{pixel:.6f}\n"
    content += "\n"  # Separate instances with a newline for clarity

# Save the content to a text file
file_path = 'Input_Bin/mnist_normalized_line_by_line.txt'
with open(file_path, 'w') as file:
    file.write(content)

print(f"Data saved to {file_path}")

Data saved to Input_Bin/mnist_normalized_line_by_line.txt


### 6.2. Convert to the Binary

In [120]:
# Read the input file, process the pixel values, and save to the output files
with open(mnist_input_file_path, 'r') as infile, \
     open(mnist_output_txt_path, 'w') as outfile_txt, \
     open(mnist_output_bat_path, 'w') as outfile_bat:

    for line in infile:
        if "Image" in line or "True class" in line:
            outfile_txt.write(line)  # Copy over the "Image" and "True class" lines to .txt
            outfile_bat.write(line)  # Copy over the "Image" and "True class" lines to .bat
        elif line.strip():  # Process pixel values if the line isn't empty
            pixel_value = float(line.strip())
            binary_rep = DtoB(pixel_value, dataWidth, weightFracWidth)  # Convert to binary
            outfile_txt.write(binary_rep + '\n')  # Write binary representation to .txt
            outfile_bat.write(binary_rep + '\n')  # Write binary representation to .bat

print(f"Binary data saved to {mnist_output_txt_path} and {mnist_output_bat_path}")

Binary data saved to Input_Bin/bin_mnist_normalized_line_by_line.txt and Input_Bin/bin_mnist_normalized_line_by_line.bat
